In [1]:
#!/usr/bin/env python3
# code to replicate the performance of the NIPS'16 MLHC workshop paper "Cam CNNS predict anatomy?"

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

os.environ["THEANO_FLAGS"] = "device=gpu,floatX=float32" 

# os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,force_device=true"#,lib.cnmem=0.7 ,nvcc.flags=-D_FORCE_INLINES"
import sys
import numpy as np
from keras import backend as K
K.set_image_dim_ordering('th')
# K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
# from Ipython.display import display, clear_output


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:
def display_length(X,Y):

#     Yr = np.round(Y)
    
    Y0 = Y[np.where(Y==0)]
    Y1 = Y[np.where(Y==1)]
    Y2 = Y[np.where(Y==2)]
    Y3 = Y[np.where(Y==3)]
    Y4 = Y[np.where(Y==4)]
    Y5 = Y[np.where(Y==5)]
    
    print('len(y=0) = ' + str(len(Y0)))
    print('len(y=1) = ' + str(len(Y1)))
    print('len(y=2) = ' + str(len(Y2)))
    print('len(y=3) = ' + str(len(Y3)))
    print('len(y=4) = ' + str(len(Y4)))
    print('len(y=5) = ' + str(len(Y5)))


In [3]:
# def Y_threshold(X,Y):

#     for i,y in enumerate(Y):
#         if y<3:
#             Y[i] = 0.0            
#         elif y>=3:
#             Y[i] = 1.0
            
#     #     Tracer()()        
# #     y_true = map(lambda x:(x==0)|(x==1),Y)
# #     Y = Y[y_true]
# #     X = X[y_true]  

#     return X,Y

In [4]:
def Y_threshold(X,Y):

    Yr = np.round(Y)
    
    Y0 = Y[np.where(Yr==0)]
    Y1 = Y[np.where(Yr==1)]
    Y2 = Y[np.where(Yr==2)]
    Y3 = Y[np.where(Yr==3)]
    Y4 = Y[np.where(Yr==4)]
    Y5 = Y[np.where(Yr==5)]
    
    print('len(y=0) = ' + str(len(Y0)))
    print('len(y=1) = ' + str(len(Y1)))
    print('len(y=2) = ' + str(len(Y2)))
    print('len(y=3) = ' + str(len(Y3)))
    print('len(y=4) = ' + str(len(Y4)))
    print('len(y=5) = ' + str(len(Y5)))
    
    for i,y in enumerate(Yr):
        if y<3:
            Yr[i] = 0.0            
        elif y>=3:
            Yr[i] = 1.0

        
    Y0 = Y[np.where(Yr==0)]
    Y1 = Y[np.where(Yr==1)]
    Y2 = Y[np.where(Yr==2)]     
    
    print('len(y=0) = ' + str(len(Y0)))
    print('len(y=1) = ' + str(len(Y1)))
    print('len(y=2) = ' + str(len(Y2)))
    
#     Tracer()()
    return X,Yr

In [5]:
# def Y_threshold(X,Y):

#     Yr = np.round(Y)
    
#     Y0 = Y[np.where(Yr==0)]
#     Y1 = Y[np.where(Yr==1)]
#     Y2 = Y[np.where(Yr==2)]
#     Y3 = Y[np.where(Yr==3)]
#     Y4 = Y[np.where(Yr==4)]
#     Y5 = Y[np.where(Yr==5)]
    
#     print('len(y=0) = ' + str(len(Y0)))
#     print('len(y=1) = ' + str(len(Y1)))
#     print('len(y=2) = ' + str(len(Y2)))
#     print('len(y=3) = ' + str(len(Y3)))
#     print('len(y=4) = ' + str(len(Y4)))
#     print('len(y=5) = ' + str(len(Y5)))
    
#     for i,y in enumerate(Yr):
#         if y<3:
#             Yr[i] = 0.0            
#         elif y==3:
#             Yr[i] = 1.0
#         elif y > 3:
#             Yr[i] = 2.0
        
#     Y0 = Y[np.where(Yr==0)]
#     Y1 = Y[np.where(Yr==1)]
#     Y2 = Y[np.where(Yr==2)]     
    
#     print('len(y=0) = ' + str(len(Y0)))
#     print('len(y=1) = ' + str(len(Y1)))
#     print('len(y=2) = ' + str(len(Y2)))
    
#     Tracer()()
#     return X,Yr

In [6]:
# def load_data05():
#     X = np.load('/work/vsankar/Project-Luna/image_array.npy')
#     Y = np.load('/work/vsankar/Project-Luna/new_labels.npy')    
    
#     X,Y = Y_threshold(X,Y)
    
#     return X,Y
    
    
    
def load_data():
    X = np.load('/work/vsankar/Project-Luna/image_array.npy')
    Y = np.load('/work/vsankar/Project-Luna/new_labels.npy')
#     Tracer()()

    
    X,Y = Y_threshold(X,Y)
    
    
    
    Yzero = Y[np.where(Y==0)]
    Xzero = X[np.where(Y==0)]

    Yone = Y[np.where(Y==1)]
    Xone = X[np.where(Y==1)]
    
#     Tracer()()
    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
    #     shear_range = 0.34,  # value in radians, equivalent to 20 deg
    #     zoom_range = [1/1.6, 1.6],   #same as in NIPS 2015 paper.
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    n1 = 2
    #increase zero dataset
#     n1 = len(Yone)/len(Yzero)
    
#     Tracer()()
    XzeroRotated , YzeroMRotated = datagen.flow(Xzero, Yzero, batch_size=16000).next()
    
#     for i in range(1):
#         XzeroM , YzeroM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
#         XzeroRotated = np.append(XzeroRotated,XzeroM,axis=0)
#         YzeroMRotated = np.append(YzeroMRotated,YzeroM,axis=0)

    #increase one dataset
#     n1 = len(Yone)/len(Yzero)  
#     XoneRotated , YoneMRotated = datagen.flow(Xone, Yone, batch_size=len(Yone)).next()
#     Tracer()()
#     for i in range(1):
#         XoneM , YoneM = datagen.flow(Xone, Yone, batch_size=len(Yone)).next()
#         XoneRotated = np.append(XoneRotated,XoneM,axis=0)
#         YoneMRotated = np.append(YoneMRotated,YoneM,axis=0)


    
    X = np.append(X,XzeroRotated,axis=0)
    Y = np.append(Y,YzeroMRotated,axis=0)
    Tracer()()
#     X = np.append(X,XoneRotated,axis=0)
#     Y = np.append(Y,YoneMRotated,axis=0)
    
#     Tracer()()
    
#     n = len(Y)/10
    
#     a = range(len(Y))
#     np.random.shuffle(a)
#     idx_test = a[:n]
#     idx_val = a[n:n*2]
#     idx_train = a[n*2:]
    
#     X_train = X[idx_train]
#     X_test = X[idx_test]
#     X_val = X[idx_val]
    
#     y_train = Y[idx_train]
#     y_test = Y[idx_test]
#     y_val = Y[idx_val]

    return X,Y


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

def startified(X,Y):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=5)
    for train_index, test_index in sss.split(X, Y):
        X_train, X_test_temp = X[train_index], X[test_index]
        Y_train, Y_test_temp = Y[train_index], Y[test_index]
    
    Tracer()()
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=5)
    for test_index, val_index in sss.split(X_test_temp, Y_test_temp):
        X_test, X_val = X_test_temp[test_index], X_test_temp[val_index]
        Y_test, Y_val = Y_test_temp[test_index], Y_test_temp[val_index]    
        
    return X_train,Y_train,X_test,Y_test,X_val,Y_val

In [8]:
X,Y  = load_data()
nb_classe=2
X_train,Y_train,X_test,Y_test,X_val,Y_val = startified(X,Y)
# X_train,Y_train,X_test,Y_test,X_val,Y_val = normalize_date(X_train,y_train,X_test,y_test,X_val,y_val,nb_classe)

len(y=0) = 3
len(y=1) = 3045
len(y=2) = 7899
len(y=3) = 12231
len(y=4) = 11153
len(y=5) = 3026
len(y=0) = 10947
len(y=1) = 26410
len(y=2) = 0


/home/vsankar/pyenvs/LungCancerDev/lib/python2.7/site-packages/ipykernel/__main__.py:68: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-6-fa86eb603506>(90)load_data()
     86 #     y_train = Y[idx_train]
     87 #     y_test = Y[idx_test]
     88 #     y_val = Y[idx_val]
     89 
---> 90     return X,Y

ipdb>  display_length(X,Y)
len(y=0) = 21894
len(y=1) = 26410
len(y=2) = 0
len(y=3) = 0
len(y=4) = 0
len(y=5) = 0
ipdb> c


/home/vsankar/pyenvs/LungCancerDev/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-7-65207ff13ea8>(10)startified()
      8 
      9     Tracer()()
---> 10     sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=5)
     11     for test_index, val_index in sss.split(X_test_temp, Y_test_temp):
     12         X_test, X_val = X_test_temp[test_index], X_test_temp[val_index]

ipdb> c


In [9]:
print('y=1')
print(len(np.where(Y_train==1)[0]))

print('y=0')
print(len(np.where(Y_train==0)[0]))

y=1
21128
y=0
17515


In [ ]:
Y_train[0]

In [ ]:
Yzero = Y_train[np.where(Y_train[:,1]==0)]
Xzero = X_train[np.where(Y_train[:,1]==0)]

Yone = Y_train[np.where(Y_train[:,1]==1)]
Xone = X_train[np.where(Y_train[:,1]==1)]
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
#     shear_range = 0.34,  # value in radians, equivalent to 20 deg
#     zoom_range = [1/1.6, 1.6],   #same as in NIPS 2015 paper.
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

#increase zero dataset
n1 = len(Yone)/len(Yzero)
# n1 = 2

XzeroRotated , YzeroMRotated = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
# Tracer()()
for i in range(n1-2):
    XzeroM , YzeroM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
    XzeroRotated = np.append(XzeroRotated,XzeroM,axis=0)
    YzeroMRotated = np.append(YzeroMRotated,YzeroM,axis=0)
    
#increase one dataset
# n1 = len(Yone)/len(Yzero)
# n_increase_zero = 2
# XoneRotated , YoneMRotated = datagen.flow(Xone, Yone, batch_size=len(Yzero)).next()
# # Tracer()()
# for i in range(n1-2):
#     XoneM , YoneM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
#     XoneRotated = np.append(XoneRotated,XoneM,axis=0)
#     YoneMRotated = np.append(YoneMRotated,YoneM,axis=0)



X_train = np.append(X_train,XzeroRotated,axis=0)
Y_train = np.append(Y_train,YzeroMRotated,axis=0)

In [10]:
print('y=1 train')
print(len(np.where(Y_train==1)[0]))

print('y=0 train')
print(len(np.where(Y_train==0)[0]))

print('y=1 test')
print(len(np.where(Y_test==1)[0]))

print('y=0 test')
print(len(np.where(Y_test==0)[0]))

print('y=1 val')
print(len(np.where(Y_val==1)[0]))

print('y=0 val')
print(len(np.where(Y_val==0)[0]))

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')

y=1 train
21128
y=0 train
17515
y=1 test
2641
y=0 test
2189
y=1 val
2641
y=0 val
2190
38643 train samples
4830 test samples
4831 val samples


In [11]:
print('X_train shape:', X_train.shape)


X_train shape: (38643, 1, 96, 96)


In [12]:
np.savez('/work/vsankar/Project-Luna/Train_all_dA_0for9k_1for0_rotated_data',X_train =  X_train,Y_train = Y_train)
np.savez('/work/vsankar/Project-Luna/Test_all_dA_0for9k_1for0_rotated_data', X_test=X_test,Y_test=Y_test)
np.savez('/work/vsankar/Project-Luna/Val_all_dA_0for9k_1for0_rotated_data', X_val=X_val,Y_val=Y_val)

In [ ]:
 data = np.load('/work/vsankar/Project-Luna/Train_nf_data.npz')

In [ ]:
data['X_train']

In [ ]:
X_train,Y_train = np.load('/work/vsankar/Project-Luna/Train_nf_data.npy')
X_test,Y_test = np.load('/work/vsankar/Project-Luna/Test_nf_data.npy')
X_val,Y_val = np.load('/work/vsankar/Project-Luna/Val_nf_data.npy')

In [ ]:
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')

In [ ]:
 np.load('/work/vsankar/Project-Luna/Train_nf_data.npy')

In [ ]:
len(np.where(y_train==1)[0])

In [ ]:
type(YzeroM)

In [ ]:
0.1*96


In [ ]:
range(0)

In [ ]:

def normalize_date(X_train,Y_train,X_test,Y_test,X_val, Y_val,nb_classes):
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    print(X_val.shape[0], 'val samples')
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('float32')
    Y_val = Y_val.astype('float32')

#     normalizing the data
#     X_train /= 4095.0   
#     X_test /= 4095.0
#     X_val /= 4095.0
    
    #std
#     X_train = X_train/np.std(X_train) - np.mean(X_train)
#     X_test = X_test/np.std(X_test) - np.mean(X_test)
    
#     Tracer()()
    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)
    Y_val = np_utils.to_categorical(Y_val, nb_classes)
    
    
    return X_train,Y_train,X_test,Y_test,X_val,Y_val